In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
import json
from typing import TypedDict, Annotated, Optional, Literal, List

from PIL import Image
import io
import base64
from dotenv import load_dotenv

load_dotenv()


True

In [33]:
def load_image(url, to_base64=False):
    if to_base64 is True:
        image_bytes = open(url, "rb").read()
        image_b64 = base64.b64encode(image_bytes).decode('utf-8')
        return image_b64
    else:
        image = Image.open(url)
        return image
    
def image_to_base64(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

In [34]:
def decode_image(b64):
    image_data = base64.b64decode(b64)
    img = Image.open(io.BytesIO(image_data))
    return img

In [44]:
def get_analysis(original_image, reference_text):
    system_prompt = f"""

    다음 예시 설명은 사용자가 입력하는 사진과 유사한 분위기의 사진을 설명한 예시다.
    예시 설명을 참고하여 사진에 대한 설명을 다음 형식으로 출력하여라.
    ### 출력 형식 : 


    ### 예시 설명 : {reference_text}

    """

    human_prompt = [
        {"type": "text", "text": " "},
        {"type": "image", "base64": original_image},
        {
            "type": "input_image",
            "image_base64": original_image
        }
    ]

    model = ChatOpenAI(model="gpt-5.1")

    messages = []
    system_message = SystemMessage(system_prompt)
    messages.append(system_message)

    human_message = HumanMessage(content=human_prompt)
    messages.append(human_message)

    response = model.invoke(messages)

    return response

In [53]:
from openai import OpenAI

def get_analysis(original_image, reference_text):
    client = OpenAI()

    system_prompt = f"""

    다음 예시 설명은 사용자가 입력하는 사진과 유사한 분위기의 사진을 설명한 예시다.
    예시 설명을 참고하여 사진에 대한 설명을 다음 형식으로 출력하여라.
    ### 출력 형식 : 


    ### 예시 설명 : {reference_text}

    """

    human_prompt = [
        {"type": "text", "text": " "},
        {'type': 'image_url', 
         'image_url': {'url': f'data:image/jpeg;base64, {original_image}'}
         }   
    ]

    messages = [
        {'role': 'system',
         'content': system_prompt},
         {'role': 'user',
          'content': human_prompt}
    ]

    response = client.chat.completions.create(
        model='gpt-5.1',
        messages=messages
    )

    return response

In [48]:
text =  """
이 욕실 인테리어는 밝고 세련된 느낌을 주고 있습니다. 화장대는 부드러운 파란색으로 칠해져 있으며, 깔끔한 흰색 대리석 또는 석재 상판이 매치되어 있습니다. 세면대 위쪽에는 긴 직사각형 모양의 거울이 설치되어 있고, 거울 양옆으로 LED 조명이 부착되어 있어 현대적인 분위기를 강조합니다. 

수도꼭지는 금속성 마감으로, 세면대의 깔끔한 디자인과 잘 어울립니다. 화장대 위에는 여러 개의 흰색 꽃병이 장식되어 있으며, 그 안에는 생화나 인조 꽃들이 꽂혀 있어 공간에 생기를 더합니다. 주변 벽은 중립적인 색조로 되어 있어 전체적인 색상 조화가 잘 이루어지고 있습니다. 

천장에는 회색의 천장 선풍기가 설치되어 있어 기능성과 함께 스타일리시한 요소를 더하고 있습니다. 전체적으로 밝고 깔끔한 느낌을 주면서도 세부적인 부분에서 세련된 디자인을 보여주는 욕실입니다.
"""

url = 'datas/images/test1.jpg'

image = image_to_base64(url)

In [54]:
response = get_analysis(image, reference_text=text)



In [55]:
response

ChatCompletion(id='chatcmpl-Ciacq3sbFBG6rPhOXfxH8qNSXvUcb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='이 세탁실 인테리어는 산뜻하고 깨끗한 느낌을 주는 공간입니다. 상·하부 장은 부드러운 민트색으로 마감되어 있어 시원하고 깔끔한 분위기를 연출하며, 화이트 상판과 잘 어우러져 밝은 인상을 줍니다. 상판 위에는 작은 싱크대와 금속성 수도꼭지가 설치되어 있어 세탁 전후 간단한 손빨래나 정리가 가능하도록 실용성을 더했습니다.\n\n벽면은 흰색 타일로 마감되어 있어 전체적으로 화사하고 정돈된 느낌을 주며, 상부장 아래에는 옷걸이 봉이 설치되어 셔츠와 옷을 걸어둘 수 있는 공간이 마련되어 있습니다. 세탁기와 건조기는 하부장 사이에 나란히 빌트인 형태로 배치되어 깔끔하고 통일감 있는 구성을 보여줍니다.\n\n왼쪽 하부장에는 라탄 바구니가 수납되어 있어 세탁물 분류나 수납에 유용하며, 상판 위에는 화분, 시계, 바구니 등의 소품이 자연스럽게 배치되어 있어 단조롭지 않고 따뜻한 분위기를 더합니다. 짙은 톤의 바닥재는 상부의 밝은 색감과 대비를 이루어 공간에 안정감을 주며, 전체적으로 아늑하면서도 실용적인 현대식 세탁실 디자인을 보여줍니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1764743256, model='gpt-5.1-2025-11-13', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=336, prompt_tokens=531, total_tokens=867, completi

In [72]:
def get_image(original_image, flower_image, flower_name, references):

    client = OpenAI()

    room_b64 = image_to_base64(original_image)
    flower_b64 = image_to_base64(flower_image)

    system_message = [
        {
            "type": "text",
            "text":
                f"다음 두 이미지를 참고해서 방 이미지에 꽃을 합성해줘. 해당 꽃의 이름은 {flower_name} 이야. 아래 references에 어디에 배치할지 참고할 정보가 포함되어 있어."
        },
    ]
    
    
    human_message = [

            # 방 이미지
            {'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64, {room_b64}'}},
            {'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64, {flower_b64}'}},
            {
                "type": "text",
                "text": f"REFERENCES: {references}"
            },
            # 생성 이미지 요청
            {
                "type": "text",
                "text": "위 두 이미지를 바탕으로 references 정보를 반영하여 정확하게 합성된 최종 인테리어 이미지를 생성해줘.",
            }
        ]
    
    messages = [
        {'role': 'system',
         'content': system_message},
        {'role': 'user',
         'content': human_message}
    ]

    image = client.images.generate(
        model='dall-e-3',
        messages=messages
    )
    return image

In [73]:
image_response = get_image('datas/images/test1.jpg', 'datas/images/images.jpeg', '산세베리아', text)

TypeError: Missing required arguments; Expected either ('prompt') or ('prompt' and 'stream') arguments to be given

In [71]:
image_response.choices[0].message.content

'지금 바로 이미지를 직접 합성해서 출력할 수는 없지만, 제공해주신 두 이미지를 바탕으로 세탁실(방) 사진에 산세베리아를 자연스럽게 합성하는 방법을 단계별로 설명해드릴게요. 포토샵이나 비슷한 편집 프로그램에서 그대로 따라 하시면 됩니다.\n\n---\n\n## 1. 배치 위치 설정 (references 반영)\n\nreferences 내용을 세탁실 이미지에 맞게 해석하면:\n\n- 밝고 세련된 느낌 유지  \n- 상판 위에 여러 꽃병/식물이 올려져 있는 구도  \n- 공간에 생기를 더하는 역할의 식물\n\n따라서 산세베리아는 **세탁실 상판 위, 수전(수도꼭지) 주변**에 배치하는 것이 가장 자연스럽습니다.\n\n**추천 위치 2곳:**\n\n1. **왼쪽 상판, 현재 보라색 난초 화분 있는 자리**  \n   - 난초 화분을 산세베리아 화분으로 대체  \n   - references의 “여러 개의 흰색 꽃병” 느낌을 살리려면, 산세베리아 뒤나 옆에 작은 흰색 액세서리나 작은 병들을 그대로 두거나 살짝 보이게 배치\n\n2. **세탁기와 싱크대 사이 상판 중앙–우측**  \n   - 원래 유칼립투스가 꽂힌 화병이 있는 위치에 산세베리아를 두고, 화병은 옆으로 살짝 이동  \n   - 세면대(싱크대) 위쪽에 식물들이 놓인 욕실 레퍼런스 구도를 재현하게 됨\n\n이 중 **1번(왼쪽 상판)** 이 가장 조화롭고 시선이 잘 모이는 위치입니다.\n\n---\n\n## 2. 산세베리아 잘라내기\n\n1. 산세베리아 이미지(두 번째 이미지)를 열기  \n2. **선택 도구** (펜툴, 퀵 셀렉션, 객체 선택 등)를 사용해  \n   - 잎 전체 + 화분만 정교하게 선택  \n   - 흰 배경은 선택에서 제외\n3. 선택 영역을 **마스크**로 만들거나,  \n   - `Ctrl/Cmd + J` 로 새 레이어에 복사해 산세베리아만 분리\n\n---\n\n## 3. 세탁실 이미지에 붙여넣기\n\n1. 분리한 산세베리아 레이어를 **세탁실 이미지** 문서로 드래그  \n2. 